<h1><center>SpaGCN Tutorial</center></h1>


<center>Author: Jian Hu,*, Xiangjie Li, Kyle Coleman, Amelia Schroeder, Nan Ma, David J. Irwin, Edward B. Lee, Russell T. Shinohara, Mingyao Li*

### 0. Installation
To install SpaGCN package you must make sure that your python version is over 3.5.=. If you don’t know the version of python you can check it by:

In [1]:
import platform
platform.python_version()

'3.8.8'

Note: Because SpaGCN pends on pytorch, you should make sure torch is correctly installed.
<br>
Now you can install the current release of SpaGCN by the following three ways:
- PyPI: Directly install the package from PyPI.

In [4]:
"""
pip3 install SpaGCN
#Note: you need to make sure that the pip is for python3，or we should install SpaGCN by
python3 -m pip install SpaGCN
pip3 install SpaGCN
#If you do not have permission (when you get a permission denied error), you should install SpaGCN by
pip3 install --user SpaGCN
"""

'\npip3 install SpaGCN\n#Note: you need to make sure that the pip is for python3，or we should install SpaGCN by\npython3 -m pip install SpaGCN\npip3 install SpaGCN\n#If you do not have permission (when you get a permission denied error), you should install SpaGCN by\npip3 install --user SpaGCN\n'

- Github
Download the package from Github and install it locally:

In [5]:
"""
git clone https://github.com/jianhuupenn/SpaGCN
cd SpaGCN/SpaGCN_package/
python3 setup.py install --user
"""

'\ngit clone https://github.com/jianhuupenn/SpaGCN\ncd SpaGCN/SpaGCN_package/\npython3 setup.py install --user\n'

- Anaconda
If you do not have Python3.5 or Python3.6 installed, consider installing Anaconda (see Installing Anaconda). After installing Anaconda, you can create a new environment, for example, SpaGCN (you can change to any name you like).

In [7]:
"""
#create an environment called SpaGCN
conda create -n SpaGCN python=3.7.9
#activate your environment 
conda activate ItClust
git clone https://github.com/jianhuupenn/SpaGCN
cd SpaGCN/SpaGCN_package/
python3 setup.py build
python3 setup.py install
conda deactivate
"""

'\n#create an environment called SpaGCN\nconda create -n SpaGCN python=3.7.9\n#activate your environment \nconda activate ItClust\ngit clone https://github.com/jianhuupenn/SpaGCN\ncd SpaGCN/SpaGCN_package/\npython3 setup.py build\npython3 setup.py install\nconda deactivate\n'

### 1. Import python modules

In [33]:
import os,csv,re
import pandas as pd
import numpy as np
import scanpy as sc
import math
import SpaGCN as spg
from scipy.sparse import issparse
import random, torch
import warnings
warnings.filterwarnings("ignore")
import matplotlib.colors as clr
import matplotlib.pyplot as plt
#Use opencv to read in image data
#!pip3 install opencv-python
import cv2
import SpaGCN as spg

In [34]:
spg.__version__

'1.0.0'

### 2. Read in data
The current version of SpaGCN requres three input data: 
<br>
1. The gene expression matrix(n by k); 
<br>
2. Spatial coordinateds of samples; 
<br>
3. Histology image(optional).
<br>
The gene expreesion data can be stored as an AnnData object. AnnData stores a data matrix .X together with annotations of observations .obs, variables .var and unstructured annotations .uns. 

In [44]:
"""
#Read original data and save it to h5ad
from scanpy import read_10x_h5
adata = read_10x_h5("../tutorial/data/expression_matrix.h5")
spatial=pd.read_csv("../tutorial/data/positions.txt",sep=",",header=None,na_filter=False,index_col=0) 
adata.obs["x1"]=spatial[1]
adata.obs["x2"]=spatial[2]
adata.obs["x3"]=spatial[3]
adata.obs["x4"]=spatial[4]
adata.obs["x5"]=spatial[5]
#Select captured samples
adata=adata[adata.obs["x1"]==1]
adata.var_names=[i.upper() for i in list(adata.var_names)]
adata.var["genename"]=adata.var.index.astype("str")
adata.write_h5ad("../tutorial/data/sample_data.h5ad")
"""
#Read in gene expression and spatial location
adata=sc.read("../tutorial/data/sample_data.h5ad")
#Read in hitology image
img=cv2.imread("../tutorial/data/histology.tif")

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


### 3. Calculate adjacent matrix

In [54]:
#Set coordinates
adata.obs["x_array"]=adata.obs["x2"]
adata.obs["y_array"]=adata.obs["x3"]
adata.obs["x_pixel"]=adata.obs["x4"]
adata.obs["y_pixel"]=adata.obs["x5"]
x_array=adata.obs["x_array"].tolist()
y_array=adata.obs["y_array"].tolist()
x_pixel=adata.obs["x_pixel"].tolist()
y_pixel=adata.obs["y_pixel"].tolist()

#Test coordinates on the image
img_new=img.copy()
for i in range(len(x_pixel)):
    x=x_pixel[i]
    y=y_pixel[i]
    img_new[int(x-20):int(x+20), int(y-20):int(y+20),:]=0

cv2.imwrite('./sample_results/151673_map.jpg', img_new)

#Calculate adjacent matrix
b=49
a=1
adj=spg.calculate_adj_matrix(x=x_pixel,y=y_pixel, x_pixel=x_pixel, y_pixel=y_pixel, image=image, beta=b, alpha=a, histology=True)
np.savetxt('./data/adj.csv', adj, delimiter=',')

Calculateing adj matrix using histology image...
Var of c0,c1,c2 =  33.30687202862215 174.55510595352243 46.84205750749746
Var of x,y,z =  5606737.526317932 4468793.817921193 5606737.526317932


### 4. Spatial domain detection using SpaGCN

#### 4.1 Expression data preprocessing

In [46]:
adata=sc.read("./data/sample_data.h5ad")
adj=np.loadtxt('./data/adj.csv', delimiter=',')
adata.var_names_make_unique()
spg.prefilter_genes(adata,min_cells=3) # avoiding all genes are zeros
spg.prefilter_specialgenes(adata)
#Normalize and take log for UMI
sc.pp.normalize_per_cell(adata)
sc.pp.log1p(adata)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


#### 4.2 Set hyper-parameters

In [47]:
#p: percentage of total expression contributed by neighborhoods.
p=0.5 

#l: parameter to control p.
#Find the l value given p, first use spg.test_l() function to get a rough estimate of the range l falls in
spg.test_l(adj,[1, 10, 100, 500, 1000])
"""
l is  1 Percentage of total expression contributed by neighborhoods: 0.0
l is  10 Percentage of total expression contributed by neighborhoods: 0.0
l is  100 Percentage of total expression contributed by neighborhoods: 0.23831094397316965
l is  500 Percentage of total expression contributed by neighborhoods: 28.014718550027993
l is  1000 Percentage of total expression contributed by neighborhoods: 153.8820492650696
"""
#Search l from 100 to 1000
l=spg.find_l(p=p,adj=adj,start=100, end=500,sep=1, tol=0.01)

#res: resolution in the initial Louvain's Clustering methods.
#If the number of clusters known, we can use the spg.search_res() fnction to search for suitable resolution(optional)
#For this toy data, we set the number of clusters=7 since this tissue has 7 layers
n_clusters=7
#Set seed
r_seed=t_seed=n_seed=100
#Seaech for suitable resolution
res=spg.search_res(adata, adj, l, n_clusters, start=0.7, step=0.1, tol=5e-3, lr=0.05, max_epochs=20, r_seed=r_seed, t_seed=t_seed, n_seed=n_seed)

l is  1 Percentage of total expression contributed by neighborhoods: 0.0
l is  10 Percentage of total expression contributed by neighborhoods: 0.0
l is  100 Percentage of total expression contributed by neighborhoods: 0.23831094397316965
l is  500 Percentage of total expression contributed by neighborhoods: 28.014718550027993
l is  1000 Percentage of total expression contributed by neighborhoods: 153.8820492650696
L= 100 P= 0.23831
L= 101 P= 0.24709
L= 102 P= 0.25606
L= 103 P= 0.2652
L= 104 P= 0.27454
L= 105 P= 0.28405
L= 106 P= 0.29376
L= 107 P= 0.30365
L= 108 P= 0.31374
L= 109 P= 0.32402
L= 110 P= 0.33449
L= 111 P= 0.34515
L= 112 P= 0.35601
L= 113 P= 0.36707
L= 114 P= 0.37832
L= 115 P= 0.38978
L= 116 P= 0.40144
L= 117 P= 0.41329
L= 118 P= 0.42536
L= 119 P= 0.43763
L= 120 P= 0.4501
L= 121 P= 0.46278
L= 122 P= 0.47567
L= 123 P= 0.48877
L= 124 P= 0.50208
Start at res =  0.7 step =  0.1
Initializing cluster centers with louvain, resolution =  0.7
Epoch  0
Epoch  10
Res =  0.7 Num of clus

#### 4.3 Run SpaGCN

In [62]:
clf=spg.SpaGCN()
clf.set_l(l)
#Set seed
random.seed(r_seed)
torch.manual_seed(t_seed)
np.random.seed(n_seed)
#Run
clf.train(adata,adj,init_spa=True,init="louvain",res=res, tol=5e-3, lr=0.05, max_epochs=200)
y_pred, prob=clf.predict()
adata.obs["pred"]= y_pred
adata.obs["pred"]=adata.obs["pred"].astype('category')
#Do cluster refinement(optional)
adj_2d=spg.calculate_adj_matrix(x=x_array,y=y_array, histology=False)
refined_pred=spg.refine(sample_id=adata.obs.index.tolist(), pred=adata.obs["pred"].tolist(), dis=adj_2d, shape="hexagon")
adata.obs["refined_pred"]=refined_pred
adata.obs["refined_pred"]=adata.obs["refined_pred"].astype('category')
#Save results
#adata.write_h5ad("./sample_results/results.h5ad")

Initializing cluster centers with louvain, resolution =  0.7
Epoch  0
Epoch  10
Epoch  20
Epoch  30
Epoch  40
Epoch  50
Epoch  60
Epoch  70
Epoch  80
Epoch  90
Epoch  100
Epoch  110
Epoch  120
Epoch  130
Epoch  140
Epoch  150
Epoch  160
Epoch  170
Epoch  180
Epoch  190
Calculateing adj matrix using xy only...


#### 4.4 Plot spatial domains

In [102]:
adata=sc.read("./sample_results/results.h5ad")
#Set colors used
plot_color=["#F56867","#FEB915","#C798EE","#59BE86","#7495D3","#D1D1D1","#6D1A9C","#15821E","#3A84E6","#997273","#787878","#DB4C6C","#9E7A7A","#554236","#AF5F3C","#93796C","#F9BD3F","#DAB370","#877F6C","#268785"]
#Plot spatial domains
domains="pred"
num_celltype=len(adata.obs[domains].unique())
adata.uns[domains+"_colors"]=list(plot_color[:num_celltype])
ax=sc.pl.scatter(adata,alpha=1,x="y_pixel",y="x_pixel",color=domains,title=domains,color_map=plot_color,show=False,size=100000/adata.shape[0])
ax.set_aspect('equal', 'box')
ax.axes.invert_yaxis()
plt.savefig("./sample_results/pred.png", dpi=600)
plt.close()

#Plot refined spatial domains
domains="refined_pred"
num_celltype=len(adata.obs[domains].unique())
adata.uns[domains+"_colors"]=list(plot_color[:num_celltype])
ax=sc.pl.scatter(adata,alpha=1,x="y_pixel",y="x_pixel",color=domains,title=domains,color_map=plot_color,show=False,size=100000/adata.shape[0])
ax.set_aspect('equal', 'box')
ax.axes.invert_yaxis()
plt.savefig("./sample_results/refined_pred.png", dpi=600)
plt.close()

**Spatial Domains**![](./sample_results/pred.png) **Refined Spatial Domains**![](./sample_results/refined_pred.png)

### 5. Identify SVGs

In [84]:
#Read in raw data
raw=sc.read("../tutorial/data/sample_data.h5ad")
raw.var_names_make_unique()
raw.obs["pred"]=adata.obs["pred"].astype('category')
raw.obs["x_array"]=raw.obs["x2"]
raw.obs["y_array"]=raw.obs["x3"]
raw.obs["x_pixel"]=raw.obs["x4"]
raw.obs["y_pixel"]=raw.obs["x5"]
#Convert sparse matrix to non-sparse
raw.X=(raw.X.A if issparse(raw.X) else raw.X)
raw.raw=raw
sc.pp.log1p(raw)

#Use domain 0 as an example
target=0
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Find neighboring domains of domain 0
#Set radius such that each spot in the target domain has approximately 10 neighbors on average
#Some potentional r values: np.quantile(adj_2d[adj_2d!=0],q=[0.003, 0.005])
r=3
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=raw.obs.index.tolist(), 
                                   x=raw.obs["x_array"].tolist(), 
                                   y=raw.obs["y_array"].tolist(), 
                                   pred=raw.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)

nbr_domians=nbr_domians[0:3]
de_genes_info=spg.rank_genes_groups(input_adata=raw,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)

de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info=de_genes_info
filtered_info=filtered_info[(filtered_info["pvals_adj"]<0.05) &
                            (filtered_info["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info["fold_change"]>min_fold_change)]
filtered_info=filtered_info.sort_values(by="in_group_fraction", ascending=False)
filtered_info["target_dmain"]=target
filtered_info["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info["genes"].tolist())

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


radius= 3 average number of neighbors for each spot is 12.524523160762943
 Cluster 0 has neighbors:
Dmain  3 :  863
Dmain  2 :  517
SVGs for domain  0 : ['CAMK2N1', 'ENC1', 'GPM6A', 'ARPP19', 'HPCAL1']


In [82]:
filtered_info

,genes,in_group_fraction,out_group_fraction,in_out_group_ratio,in_group_mean_exp,out_group_mean_exp,fold_change,pvals_adj,target_dmain,neighbors
0,CAMK2N1,1.000000,0.944964,1.058242,2.333675,1.578288,2.128434,1.656040e-11,0,"[3, 2]"
2,ENC1,0.998638,0.941848,1.060295,2.457791,1.696083,2.141931,1.552131e-03,0,"[3, 2]"
4,GPM6A,0.997275,0.922118,1.081505,2.224006,1.561187,1.940255,8.602227e-03,0,"[3, 2]"
6,ARPP19,0.982289,0.853583,1.150784,1.889256,1.272106,1.853637,4.823349e-02,0,"[3, 2]"
1,HPCAL1,0.851499,0.465213,1.830342,1.141321,0.406338,2.085448,9.706465e-05,0,"[3, 2]"


In [103]:
#Plot refinedspatial domains
color_self = clr.LinearSegmentedColormap.from_list('pink_green', ['#3AB370',"#EAE7CC","#FD1593"], N=256)
for g in filtered_info["genes"].tolist():
    raw.obs["exp"]=raw.X[:,raw.var.index==g]
    ax=sc.pl.scatter(raw,alpha=1,x="y_pixel",y="x_pixel",color="exp",title=g,color_map=color_self,show=False,size=100000/raw.shape[0])
    ax.set_aspect('equal', 'box')
    ax.axes.invert_yaxis()
    plt.savefig("./sample_results/"+g+".png", dpi=600)
    plt.close()


**CAMK2N1**![](./sample_results/CAMK2N1.png) **ENC1**![](./sample_results/ENC1.png) **GPM6A**![](./sample_results/GPM6A.png) **ARPP19**![](./sample_results/ARPP19.png) **HPCAL1**![](./sample_results/HPCAL1.png)

### 6. Identify Meta Gene

In [132]:
#Use domain 2 as an example
target=2
meta_name, meta_exp=spg.find_meta_gene(input_adata=raw,
                    pred=raw.obs["pred"].tolist(),
                    target_domain=target,
                    start_gene="GFAP",
                    mean_diff=0,
                    early_stop=True,
                    max_iter=3,
                    use_raw=False)

raw.obs["meta"]=meta_exp

Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.


Add gene:  MGP
Minus gene:  FTH1
Absolute mean change: 0.8913243
Number of non-target spots reduced to: 1888
Meta gene is:  GFAP+MGP-FTH1


Trying to set attribute `.obs` of view, copying.


Add gene:  MYL9
Minus gene:  MBP
Absolute mean change: 2.175557
Number of non-target spots reduced to: 563
Meta gene is:  GFAP+MGP-FTH1+MYL9-MBP
Add gene:  KRT8
Minus gene:  MT-ATP6
Absolute mean change: 2.8935516
Number of non-target spots reduced to: 111
Meta gene is:  GFAP+MGP-FTH1+MYL9-MBP+KRT8-MT-ATP6


In [133]:
#Plot meta gene
g="GFAP"
raw.obs["exp"]=raw.X[:,raw.var.index==g]
ax=sc.pl.scatter(raw,alpha=1,x="y_pixel",y="x_pixel",color="exp",title=g,color_map=color_self,show=False,size=100000/raw.shape[0])
ax.set_aspect('equal', 'box')
ax.axes.invert_yaxis()
plt.savefig("./sample_results/"+g+".png", dpi=600)
plt.close()

raw.obs["exp"]=raw.obs["meta"]
ax=sc.pl.scatter(raw,alpha=1,x="y_pixel",y="x_pixel",color="exp",title=meta_name,color_map=color_self,show=False,size=100000/raw.shape[0])
ax.set_aspect('equal', 'box')
ax.axes.invert_yaxis()
plt.savefig("./sample_results/meta_gene.png", dpi=600)
plt.close()


**start**![](./sample_results/GFAP.png) **meta gene**![](./sample_results/meta_gene.png)